In [1]:
from elasticsearch import Elasticsearch

# Thay thế cloud_id bằng cloud_id của bạn
# url ="https://booksearch.es.us-central1.gcp.cloud.es.io/"
# cloud_id = "booksearch:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvJDg0NjljZWUyZGI2MjRkNmZiYzA0ZjM2NjJmYjgxMjIyJGMyN2Y2YzE4OGYxMzQ5ZDFhMzRmZDE4NjY3MjU0OWIy"
username = "elastic"
password = "BqMnkSKwGlpBv3QlqMl7qY1k"
# Tạo đối tượng Elasticsearch

es = Elasticsearch(
    cloud_id="15858ac8d7ab4d099dad7912514e73df:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvOjQ0MyQ5ZTBlZjkxZTQxNWM0NGU0OTdiZDQ4NDg3NDA5OWU5MiQ0MWM0ZjBhMmJlODc0MTc1OTFjMzUyNGYzNmYzN2I2MQ==",
    basic_auth=(username, password)
)
print(es.ping)



<bound method Elasticsearch.ping of <Elasticsearch(['https://9e0ef91e415c44e497bd484874099e92.us-central1.gcp.cloud.es.io:443'])>>


In [2]:
#Read dataset and cleaning data
import pandas as pd

# Load the dataset
df = pd.read_csv("GoogleBookAPIDataset.csv")

# Drop unnecessary columns
df = df.drop(columns=['Unnamed: 0'])

# Handle duplicates
df = df.drop_duplicates()

# Standardize column names
df.columns = df.columns.str.lower().str.replace(' ', '_')

# Handle missing values
df = df.dropna()

print(df.columns)



Index(['unnamed:_0.1', 'id', 'title', 'desc', 'authors', 'categories',
       'averagerating', 'maturityrating', 'publisheddate', 'pagecount'],
      dtype='object')


In [5]:
# delete index
es.indices.delete(index="books", ignore=[400, 404])

C:\Users\DELL\AppData\Local\Temp\ipykernel_18212\525162218.py:2: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.delete(index="books", ignore=[400, 404])


ObjectApiResponse({'error': {'root_cause': [{'type': 'index_not_found_exception', 'reason': 'no such index [books]', 'index_uuid': '_na_', 'resource.type': 'index_or_alias', 'resource.id': 'books', 'index': 'books'}], 'type': 'index_not_found_exception', 'reason': 'no such index [books]', 'index_uuid': '_na_', 'resource.type': 'index_or_alias', 'resource.id': 'books', 'index': 'books'}, 'status': 404})

In [1]:
#original
#mapping step
mappings = {
    #tokenizing of data to words
    "properties":{
        "id":{"type":"keyword"},
        "title":{"type":"text", "analyzer":"english"},
        "desc":{"type":"text", "analyzer":"english"},
        "authors":{"type":"keyword"},
        "categories": {"type": "keyword"},
        "averageRating": {"type": "float"},
        "maturityRating": {"type": "keyword"},
        "publishedDate": {"type": "date"},
        "pageCount": {"type": "integer"}
    }
}

# create index books
es.indices.create(index="books", mappings=mappings)

NameError: name 'es' is not defined

In [4]:
#test
#mapping step
mappings = {
    #tokenizing of data to words
    "properties":{
        "id":{"type":"keyword"},
        "title":{"type":"text", "analyzer":"english"},
        "desc":{"type":"dense-vector", "dims": 3, "analyzer":"english"},
        "authors":{"type":"keyword"},
        "categories": {"type": "keyword"},
        "averageRating": {"type": "float"},
        "maturityRating": {"type": "keyword"},
        "publishedDate": {"type": "date"},
        "pageCount": {"type": "integer"}
    }
}

# create index books
es.indices.create(index="books", mappings=mappings)

BadRequestError: BadRequestError(400, 'mapper_parsing_exception', 'No handler for type [dense-vector] declared on field [desc]')

In [36]:
# indexed documents
for i, row in df.iterrows():
    doc = {
        "id": row["id"],
        "title": row["title"],
        "desc": row["desc"],
        "authors": row["authors"],
        "categories": row["categories"],
        "averagerating": row["averagerating"],
        "maturityrating": row["maturityrating"],
        "publisheddate": row["publisheddate"],
        "pagecount": row["pagecount"]
    }
            
    es.index(index="books", id=i, document=doc)

KeyboardInterrupt: 

In [9]:
# another way to store data to index
from elasticsearch.helpers import bulk

bulk_data = []
for i,row in df.iterrows():
    bulk_data.append(
        {
            "_index": "books",
            "_id": row["id"],
            "_source": {        
                "id": row["id"],
        "title": row["title"],
        "desc": row["desc"],
        "authors": row["authors"],
        "categories": row["categories"],
        "averagerating": row["averagerating"],
        "maturityrating": row["maturityrating"],
        "publisheddate": row["publisheddate"],
        "pagecount": row["pagecount"],
            }
        }
    )
bulk(es, bulk_data)

(22332, [])

In [10]:
# ounting the number of items in the index:
es.indices.refresh(index="books")
es.cat.count(index="books", format="json")

ListApiResponse([{'epoch': '1704208599', 'timestamp': '15:16:39', 'count': '22332'}])

In [11]:
search_term = input("Enter the title you want to search: ")
resp = es.search(
    index="books",
    query={
            "bool": {
                "must": {
                    "match_phrase": {
                        "title": search_term,
                    }
                },
                
            },
        },            
)
print(resp.body)
# Extract hits from the response
hits = resp['hits']['hits']

# Extract relevant information from each hit
data = [{'id': hit['_source']['id'],
         'title': hit['_source']['title'],
         'desc': hit['_source']['desc'],
         'authors': hit['_source']['authors'],
         'categories': hit['_source']['categories'],
         'averagerating': hit['_source']['averagerating'],
         'maturityrating': hit['_source']['maturityrating'],
         'publisheddate': hit['_source']['publisheddate'],
         'pagecount': hit['_source']['pagecount']} for hit in hits]

# Create a DataFrame
df_result = pd.DataFrame(data)

try:
    existing_df = pd.read_csv('result.csv')
except FileNotFoundError:
    existing_df = pd.DataFrame()

    # Append new results to the existing DataFrame
updated_df = pd.concat([existing_df, df_result], ignore_index=True)

    # Save the updated DataFrame to 'result.csv' (overwrite)
updated_df.to_csv('result.csv', index=False)
print("Results appended to result.csv")

{'took': 4, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 81, 'relation': 'eq'}, 'max_score': 6.396386, 'hits': [{'_index': 'books', '_id': 'k2N0PwAACAAJ', '_score': 6.396386, '_source': {'id': 'k2N0PwAACAAJ', 'title': 'Queen of Sorcery', 'desc': 'Legends tell how Belgarath the sorcerer and his daughter Polgara defeated the evil God Torak, imprisoning him in an endless sleep. But now a priest of Torak is racing to his God with the Orb of Aldur and is racing to reawaken him. Belgarath and Polgara are on his trail. With them is Garion, a simple farm boy only months before. And with each league the group travel, the power of sorcery is growing in Garon . . .', 'authors': "['David Eddings']", 'categories': "['Fantasy']", 'averagerating': 3.5, 'maturityrating': 'NOT_MATURE', 'publisheddate': '2006', 'pagecount': 446}}, {'_index': 'books', '_id': '64krE5BGr3gC', '_score': 6.396386, '_source': {'id': '64krE5BGr3gC', 'title